In [1]:
import pandas as pd
import requests
import os
from datetime import datetime

In [206]:
API_KEY = os.environ["FRED_API_KEY"]

In [207]:
def fetch_fred_data(series_id, start_date, end_date):
   
    # FRED API URL
    url = f'https://api.stlouisfed.org/fred/series/observations'

    # Parameters for the API request
    params = {
        'series_id': series_id,
        'api_key': API_KEY,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
    }

    # Sending the request to the FRED API
    response = requests.get(url, params=params)
    data = response.json()

    # Check for errors
    if 'observations' not in data:
        print(f"Error fetching data for {series_id}: {data.get('error_message', 'Unknown error')}")
        return pd.DataFrame()

    # Extract observations
    observations = data['observations']
    
    # Convert to DataFrame
    df = pd.DataFrame(observations)
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')  # Convert values to numeric

    # Select relevant columns
    df = df[['date', 'value']]
    
    return df

In [208]:
#Define Date Range
start_date = '2019-08-01'
end_date = '2024-08-01'

In [209]:
#Fetch Unemployment Rate (UNRATE) data
unemployment_series_id = 'UNRATE' 

# Fetch data
fetched_unemployment_data = fetch_fred_data(unemployment_series_id, start_date, end_date)


In [210]:
#Fetch Consumer Price Index for All Urban Consumers: All Items in U.S. City Average (CPIAUSCL) data from 2020-01-01 to 2024-01-01
CPI_series_id = 'CPIAUCSL'

# Fetch data
fetched_cpi_data = fetch_fred_data(CPI_series_id, start_date, end_date)

In [211]:
#Fetch 3 Month Treasury Bill Rates
threemonth_tbill_series_id = 'TB3MS'

# Fetch data
fetched_threemonth_tbill_data = fetch_fred_data(threemonth_tbill_series_id, start_date, end_date)

In [212]:
#Fetch 6 Month Treasury Bill Rates
sixmonth_tbill_series_id = 'TB6MS'

# Fetch data
fetched_sixmonth_tbill_data = fetch_fred_data(sixmonth_tbill_series_id, start_date, end_date)

In [213]:
#Fetch 6 Month Treasury Bill Rates
sixmonth_tbill_series_id = 'TB6MS'

# Fetch data
fetched_sixmonth_tbill_data = fetch_fred_data(sixmonth_tbill_series_id, start_date, end_date)

In [214]:
#Fetch BBB ICE BofA Index Effective Yield
BBB_series_id = 'BAMLC0A4CBBBEY'

# Fetch data
fetched_BBB_data = fetch_fred_data(BBB_series_id, start_date, end_date)

In [215]:
#Data Cleaning
unemployment_data = fetched_unemployment_data.set_index('date')
unemployment_data.index = pd.to_datetime(unemployment_data.index)
unemployment_data = unemployment_data.rename(columns={'value': 'Unemployment Rate'})

cpi_data = fetched_cpi_data.set_index('date')
cpi_data.index = pd.to_datetime(cpi_data.index)
cpi_data = cpi_data.rename(columns={'value': 'CPI'})

threemonth_tbill_data = fetched_threemonth_tbill_data.set_index('date')
threemonth_tbill_data.index = pd.to_datetime(threemonth_tbill_data.index)
threemonth_tbill_data = threemonth_tbill_data.rename(columns={'value': 'Three Month T Bill Rate'})

sixmonth_tbill_data = fetched_sixmonth_tbill_data.set_index('date')
sixmonth_tbill_data.index = pd.to_datetime(sixmonth_tbill_data.index)
sixmonth_tbill_data = sixmonth_tbill_data.rename(columns={'value': 'Six Month T Bill Rate'})

BBB_data = fetched_BBB_data.set_index('date')
BBB_data.index = pd.to_datetime(BBB_data.index)
BBB_data = BBB_data.resample('MS').first()
BBB_data = BBB_data.rename(columns={'value': 'BBB Rate'})

In [216]:
# Merge dataframes together one by one
merged_data = pd.merge(unemployment_data, cpi_data, on='date')
merged_data = pd.merge(merged_data, threemonth_tbill_data, on='date')
merged_data = pd.merge(merged_data, sixmonth_tbill_data, on='date')
merged_data = pd.merge(merged_data, BBB_data, on='date')


In [217]:
merged_data

,Unemployment Rate,CPI,Three Month T Bill Rate,Six Month T Bill Rate,BBB Rate
date,,,,,
2019-01-01,4.0,252.561,2.37,2.45,4.67
2019-02-01,3.8,253.319,2.39,2.44,4.42
2019-03-01,3.8,254.277,2.40,2.44,4.39
2019-04-01,3.7,255.233,2.38,2.39,4.10
2019-05-01,3.6,255.296,2.35,2.35,3.98
...,...,...,...,...,...
2024-03-01,3.8,312.230,5.24,5.11,5.61
2024-04-01,3.9,313.207,5.24,5.15,5.66
2024-05-01,4.0,313.225,5.25,5.17,5.92
